In [ ]:
## Packages importing
import numpy as np
import pandas as pd
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
## Printing variable info func
def getInfo(val):
    print(type(val))
    try:
        print(val)
        if str(type(val)) == "<class 'numpy.ndarray'>" or \
           str(type(val)) == "<class 'pandas.core.frame.DataFrame'>":
            print(val.shape)
        else:
            print(len(val))
    except Exception:
        pass


# Plotting func
def plot_acc(h, title='accuracy'):
    plt.plot(h.history['accuracy'])
    plt.plot(h.history['val_accuracy'])
    plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc=0)

def plot_loss(h, title='loss'):
    plt.plot(h.history['loss'])
    plt.plot(h.history['val_loss'])
    plt.title(title)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Validation'], loc=0)

def plot_loss_acc(h, title='accuracy & loss'):
    plt.plot(h.history['accuracy'])
    plt.plot(h.history['val_accuracy'])
    plt.plot(h.history['loss'])
    plt.plot(h.history['val_loss'])
    plt.title(title)
    plt.ylabel('Accuract & Loss')
    plt.xlabel('Epoch')
    plt.legend(['Training_acc', 'Validation_acc', 'Training_loss', 'Validation_loss'], loc=0)
    plt.show()
    plt.clf()

In [ ]:
## Data Preparation for wdbc.data (Not Used)

# data = pd.read_csv("./hw2_data/wdbc.data", header=None)
# # getInfo(data)

# # Data Preparation
# data = data.drop(data.columns[0], axis=1)                               # remove ID col
# data = data.astype(str)
# data[data.columns[0]] = data[data.columns[0]].map({'M': 1, 'B': 0})     # 2 for benign(0), 4 for malignant(1)
# # getInfo(data)

# x = data[data.columns[2:12]].values
# getInfo(x)
# y = data[data.columns[0]].values
# getInfo(y)

# # Data normalization
# scaler = MinMaxScaler()
# x = scaler.fit_transform(x)
# getInfo(x)

먼저, pandas library 의 read_csv()를 통해 제공된 데이터를 읽어왔습니다.
그런데 breast-cancer-wisconsin.data 의 경우 첫번째 line부터 바로 raw data가 있고, 이 값들이 DataFrame의 col이 되어 첫 줄의 Data가 누락되는 현상을 방지하기 위해 header의 parameter 값을 None으로 설정해 주었습니다.

이 데이터에 대한 설명을 제공하는 breast-cancer-wisconsin.names라는 파일의 "7. Attribute information" 부분을 주목하면,

1번째 col은 ID, 2번째부터 10번째 col은 각각의 데이터가 보관되어 있으며
그리고 마지막 11번째 col은 양성인지 악성인지 판별하는 class (output) 이 보관되어 있다고 명시되어 있습니다.

때문에 이번 학습에 사용되지 않는 첫번째 col 의 값을 drop() 를 이용해 제거했으며, 2번째부터 10번째까지의 data들을 input(x), 11번째(마지막)의 data들을 output(y) 라고 하였습니다.

그런데 마지막 col 에서 benign는 2, malignant는 4라고 저장되어 있기에 DataFrame type의 array를 map()을 이용해 0 또는 1로 새롭게 mapping 해주었습니다.


다음으로 missing value 처리를 위해 "?"가 포함된 row를 replace()를 통해 np.nan으로 변경하고 dropna() 수행했습니다.
결과적으로 16개의 sample을 제거해서 699개에서 683개의 샘플로 줄은 것을 확인할 수 있습니다.

그 이후 MinMaxScaler를 이용해 normalizing 과정을 거쳤으며 반환된 numpy.ndarray의 변수를 제시한 조건에 맞게 split 시켜서 test, validation, train 을 위한 데이터를 구분했습니다.

In [ ]:
## Data Preparation

# Get RAW Data
data = pd.read_csv("./hw2_data/breast-cancer-wisconsin.data", header=None)
# getInfo(data)

data = data.drop(data.columns[0], axis=1)                               # remove ID col
data = data.astype(str)
data = data.replace(['?'], np.nan).dropna()                             # Missing Value Processing
data[data.columns[-1]] = data[data.columns[-1]].map({'4': 1, '2': 0})   # 2 for benign(0), 4 for malignant(1)
# getInfo(data)

x = data[data.columns[:-1]].values
# getInfo(x)
y = data[data.columns[-1]].values
# getInfo(y)

# Data normalization
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
# getInfo(x)

# Test Validation Data split
x_test = x[0:100]
y_test = y[0:100]
x_val = x[100:200]
y_val = y[100:200]
x_train = x[200:]
y_train = y[200:]

NN의 modeling과 train, evaluate를 모두 수행한 후 학습 결과를 저장한 ndarray type의 array를 반환하는 함수입니다.
뒤의 과정(Q2~Q4) 에 여러 trial을 거쳐서 학습을 수행해야 했기 때문에, trials 를 parameter 로 받고 반복 횟수를 지정하여 학습할 수 있도록 하였으며,
hl_activation과 hl_neurons 또한 parameter로 받아서 modeling 할 때 activation function과 hidden neuron의 개수도 지정할 수 있도록 하였습니다.

이 함수는 package import 아래가 아닌 중간 부분에 선언되었는데,
이미 input data는 x로 정해져 있고, 불필요한 Preparation 과정을 막고자 input data를 parameter에 넣지 않고 global variable로 처리하여 input data를 주었습니다.

초기에 학습 결과를 저장할 result라는 각각의 label만 존재하는 빈 array를 선언하였고, for 문을 사용하여 argument로 받아온 trial만큼 반복을 수행하도록 하였습니다.

callbacks의 옵션을 주어 early-stopping 할 수 있도록 overfitting을 방지했으며 실제로 early-stopping을 하지 않고 모든 epoch 200회를 채웠을 때보다 accuracy가 더 좋게 나온 것을 확인했습니다.

문제에서 요구한 대로 2 epoch 동안 val_loss의 개선이 없으면 중단되도록 patience의 값을 2로 주었으며 불필요한 epoch 진행과정을 보여주는 것을 생략하기 위해 verbose 의 값을 0으로 주었습니다.

train이 모두 끝난 후 결과를 리턴하기 위해서 epoch가 끝나기 직전 history에 저장된 마지막 번째의 loss 값과 accuracy의 값을 사용했으며,
한 번의 trial 이 끝나고 만들어진 train set 과 test set의 metrics들을 출력시키고, res라는 ndarray 에 담아 초기에 선언한 result에 append 시켰습니다.


In [ ]:
def train_and_evaluate(trials=1, hl_activation='relu', hl_neurons=10):
    
    # Empty Table for result
    result=np.array(["Training loss", "Training accuracy", "Test loss", "Test accuracy"]).reshape(1, 4)
    
    for trial in range(trials):
      # Neural Network Modeling
      model = models.Sequential()
      model.add(layers.Dense(hl_neurons, activation=hl_activation, input_shape=(9,))) # input layer & HL
      model.add(layers.Dense(1, activation='sigmoid'))                                # output layer
      model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
      
      # Training w/o early-stopping
      # history = model.fit(x_train, y_train, 
      #                     epochs=200, batch_size=10, validation_data=(x_val, y_val), verbose=0)

      # Training w/ early-stopping
      history = model.fit(x_train, y_train, 
                              epochs=200, batch_size=10, validation_data=(x_val, y_val), 
                              callbacks = [EarlyStopping(monitor='val_loss', patience=2)], verbose=0)

      # Evaluating with test data
      test_loss, test_acc = model.evaluate(x_test,y_test)

      # Visualization
      print("Trial #", trial+1)
      print("Training loss=", history.history['loss'][-1],\
            "\nTraining accuracy=", history.history['accuracy'][-1],\
            "\nTest loss=", test_loss,\
            "\nTest accuracy=", test_acc)      
      res = np.array([history.history['loss'][-1], history.history['accuracy'][-1], test_loss, test_acc]).reshape(1, 4)
      result = np.append(result, res, axis=0)
      
      # Plotting
      plot_loss_acc(history)
    return result

각각의 column 에 대해 평균과 표준편차를 구하는 함수입니다.

맨 첫 번째 row에는 label 이 담겨있으므로 이것을 제외한 나머지 row 를 선택하고 for 문에서 iter로 사용된 col로 원하는 column을 선택했으므로 ndarray[1:, col] 라고 하였으며
이것을 float type로 바꾸어주어 평균과 표준편차를 구하기 위한 연산을 할 수 있게 하였습니다.

In [ ]:
def getMean_SD(ndarray):
    # iterate with # of col
    for col in range(ndarray.shape[1]):
        mean = np.mean(ndarray[1:, col].astype('float32'))
        std = np.std(ndarray[1:, col].astype('float32'))
        print(ndarray[0, col], "\nMean=", mean, "SD=", std)

In [ ]:
# Q2
Q2 = train_and_evaluate(5)

In [ ]:
# Q3 (Affection of HL activation function)
hl_activation = [None, 'relu', 'sigmoid', 'tanh']
Q3 = [getMean_SD(train_and_evaluate(trials=10, hl_activation=activation, hl_neurons=10)) for activation in hl_activation]

In [ ]:
# Q4 (Affection of HL Neurons #, roughly)
hl_neurons = [2, 5, 10, 20, 50, 100, 200, 500, 1000, 2000]
Q4 = [getMean_SD(train_and_evaluate(trials=5, hl_activation='relu', hl_neurons=neurons)) for neurons in hl_neurons]

In [ ]:
# QE1 (Affection of HL Neurons #, fine tuned)
hl_neurons = [25, 30, 35, 40, 45]
QE1 = [getMean_SD(train_and_evaluate(trials=1, hl_activation='relu', hl_neurons=neurons)) for neurons in hl_neurons]

In [ ]:
# QE2
def getWeight(trials=1): 
    # Neural Network Modeling
    model = models.Sequential()
    model.add(layers.Dense(1, input_shape=(9,), activation='sigmoid')) # input & output layer
    model.compile(optimizer='rmsprop',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
    
    # Training w/ early-stopping
    history = model.fit(x_train, y_train, 
                            epochs=200, batch_size=10, validation_data=(x_val, y_val), 
                            callbacks = [EarlyStopping(monitor='val_loss', patience=2)], verbose=0)

    # Evaluating with test data
    test_loss, test_acc = model.evaluate(x_test,y_test)

    # Get weight and bias
    w = model.get_weights()[0]
    b = model.get_weights()[1]
    return w, b

In [ ]:
(weight, bias) = getWeight()
getInfo(weight)
getInfo(bias)